<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 6.6
## Feature Encoding

### Data

**Predict the chronic kidney disease.**

This dataset can be used to predict the chronic kidney disease and it can be collected from the hospital nearly 2 months of period.

We use the following representation to collect the dataset 
- age	-	age	
- bp	-	blood pressure 
- sg	-	specific gravity 
- al	- albumin 
- su	-	sugar 
- rbc	-	red blood cells 
- pc	-	pus cell 
- pcc	-	pus cell clumps 
- ba	-	bacteria 
- bgr	-	blood glucose random 
- bu	-	blood urea 
- sc	-	serum creatinine 
- sod	-	sodium 
- pot	-	potassium 
- hemo	-	hemoglobin 
- pcv	-	packed cell volume 
- wc	-	white blood cell count 
- rc	-	red blood cell count 
- htn	-	hypertension 
- dm	-	diabetes mellitus 
- cad	-	coronary artery disease 
- appet	-	appetite 
- pe	-	pedal edema 
- ane	-	anemia 
- class	-	class




[Chronic Kidney Disease DataSet](https://archive.ics.uci.edu/ml/datasets/chronic_kidney_disease)

In [207]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_selection import chi2, SelectKBest, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

%matplotlib inline
plt.style.use('ggplot')

#### 1. Load Data

In [5]:
# Read Data
kidney_disease_csv = 'kidney_disease.csv'

In [34]:
df = pd.read_csv(kidney_disease_csv)
df.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


#### 2. Perform EDA

Perform EDA. Check null values. Impute if necessary.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              400 non-null    int64  
 1   age             391 non-null    float64
 2   bp              388 non-null    float64
 3   sg              353 non-null    float64
 4   al              354 non-null    float64
 5   su              351 non-null    float64
 6   rbc             248 non-null    object 
 7   pc              335 non-null    object 
 8   pcc             396 non-null    object 
 9   ba              396 non-null    object 
 10  bgr             356 non-null    float64
 11  bu              381 non-null    float64
 12  sc              383 non-null    float64
 13  sod             313 non-null    float64
 14  pot             312 non-null    float64
 15  hemo            348 non-null    float64
 16  pcv             330 non-null    object 
 17  wc              295 non-null    obj

In [35]:
df = df.set_index('id')

In [12]:
df

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
id,,,,,,,,,,,,,,,,,,,,,
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,140.0,...,47,6700,4.9,no,no,no,good,no,no,notckd
396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,...,54,7800,6.2,no,no,no,good,no,no,notckd
397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,100.0,...,49,6600,5.4,no,no,no,good,no,no,notckd


In [44]:
for column in df.columns:
    print(df[column].value_counts(dropna=False))

60.0    19
65.0    17
48.0    12
55.0    12
50.0    12
        ..
83.0     1
27.0     1
3.0      1
11.0     1
81.0     1
Name: age, Length: 77, dtype: int64
80.0     116
70.0     112
60.0      71
90.0      53
100.0     25
NaN       12
50.0       5
110.0      3
120.0      1
140.0      1
180.0      1
Name: bp, dtype: int64
1.020    106
1.010     84
1.025     81
1.015     75
NaN       47
1.005      7
Name: sg, dtype: int64
0.0    199
NaN     46
1.0     44
2.0     43
3.0     43
4.0     24
5.0      1
Name: al, dtype: int64
0.0    290
NaN     49
2.0     18
3.0     14
1.0     13
4.0     13
5.0      3
Name: su, dtype: int64
normal      201
NaN         152
abnormal     47
Name: rbc, dtype: int64
normal      259
abnormal     76
NaN          65
Name: pc, dtype: int64
notpresent    354
present        42
NaN             4
Name: pcc, dtype: int64
notpresent    374
present        22
NaN             4
Name: ba, dtype: int64
NaN      44
99.0     10
100.0     9
93.0      9
107.0     8
         ..
307.0 

In [45]:
df.dtypes #pcv wc rc to numeric

age               float64
bp                float64
sg                float64
al                float64
su                float64
rbc                object
pc                 object
pcc                object
ba                 object
bgr               float64
bu                float64
sc                float64
sod               float64
pot               float64
hemo              float64
pcv                object
wc                 object
rc                 object
htn                object
dm                 object
cad                object
appet              object
pe                 object
ane                object
classification     object
dtype: object

In [47]:
df.dm.value_counts()

no       258
yes      134
\tno       3
\tyes      2
 yes       1
Name: dm, dtype: int64

In [57]:
test = df.dm.replace(['\tno','\tyes',' yes'],['no','yes','yes'])

In [58]:
test.value_counts()

no     261
yes    137
Name: dm, dtype: int64

In [59]:
df.dm = test

In [60]:
df.dm.value_counts()

no     261
yes    137
Name: dm, dtype: int64

In [61]:
df.cad = df.cad.replace('\tno','no')
df.cad.value_counts()

no     364
yes     34
Name: cad, dtype: int64

In [62]:
df.classification = df.classification.replace('ckd\t','ckd')
df.classification.value_counts()

ckd       250
notckd    150
Name: classification, dtype: int64

In [63]:
for column in df.columns:
    print(df[column].value_counts(dropna=False))

60.0    19
65.0    17
48.0    12
55.0    12
50.0    12
        ..
83.0     1
27.0     1
3.0      1
11.0     1
81.0     1
Name: age, Length: 77, dtype: int64
80.0     116
70.0     112
60.0      71
90.0      53
100.0     25
NaN       12
50.0       5
110.0      3
120.0      1
140.0      1
180.0      1
Name: bp, dtype: int64
1.020    106
1.010     84
1.025     81
1.015     75
NaN       47
1.005      7
Name: sg, dtype: int64
0.0    199
NaN     46
1.0     44
2.0     43
3.0     43
4.0     24
5.0      1
Name: al, dtype: int64
0.0    290
NaN     49
2.0     18
3.0     14
1.0     13
4.0     13
5.0      3
Name: su, dtype: int64
normal      201
NaN         152
abnormal     47
Name: rbc, dtype: int64
normal      259
abnormal     76
NaN          65
Name: pc, dtype: int64
notpresent    354
present        42
NaN             4
Name: pcc, dtype: int64
notpresent    374
present        22
NaN             4
Name: ba, dtype: int64
NaN      44
99.0     10
100.0     9
93.0      9
107.0     8
         ..
307.0 

#### Impute Null Values

Impute null values for numeric and object columns.

In [104]:
df.loc[:,'rc'] = pd.to_numeric(df.rc,errors='coerce')
df.loc[:,'pcv'] = pd.to_numeric(df.pcv,errors='coerce')
df.loc[:,'wc'] = pd.to_numeric(df.wc,errors='coerce')

In [105]:
imputer = SimpleImputer(strategy='most_frequent')

In [106]:
imputed_df = imputer.fit_transform(df)

In [107]:
df.shape

(400, 25)

In [108]:
columns = df.columns

In [109]:
mapper = {}

for i in range(len(columns)):
    mapper[i] = columns[i]
mapper

{0: 'age',
 1: 'bp',
 2: 'sg',
 3: 'al',
 4: 'su',
 5: 'rbc',
 6: 'pc',
 7: 'pcc',
 8: 'ba',
 9: 'bgr',
 10: 'bu',
 11: 'sc',
 12: 'sod',
 13: 'pot',
 14: 'hemo',
 15: 'pcv',
 16: 'wc',
 17: 'rc',
 18: 'htn',
 19: 'dm',
 20: 'cad',
 21: 'appet',
 22: 'pe',
 23: 'ane',
 24: 'classification'}

In [110]:
imputed_df = pd.DataFrame(imputed_df)
imputed_df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,48.0,80.0,1.02,1.0,0.0,normal,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.02,4.0,0.0,normal,normal,notpresent,notpresent,99.0,...,38.0,6000.0,5.2,no,no,no,good,no,no,ckd
2,62.0,80.0,1.01,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,5.2,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.01,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.02,0.0,0.0,normal,normal,notpresent,notpresent,140.0,...,47.0,6700.0,4.9,no,no,no,good,no,no,notckd
396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,...,54.0,7800.0,6.2,no,no,no,good,no,no,notckd
397,12.0,80.0,1.02,0.0,0.0,normal,normal,notpresent,notpresent,100.0,...,49.0,6600.0,5.4,no,no,no,good,no,no,notckd
398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,...,51.0,7200.0,5.9,no,no,no,good,no,no,notckd


In [111]:
renamed_df = imputed_df.rename(mapper=mapper,axis=1)
renamed_df

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.02,1.0,0.0,normal,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.02,4.0,0.0,normal,normal,notpresent,notpresent,99.0,...,38.0,6000.0,5.2,no,no,no,good,no,no,ckd
2,62.0,80.0,1.01,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,5.2,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.01,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.02,0.0,0.0,normal,normal,notpresent,notpresent,140.0,...,47.0,6700.0,4.9,no,no,no,good,no,no,notckd
396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,...,54.0,7800.0,6.2,no,no,no,good,no,no,notckd
397,12.0,80.0,1.02,0.0,0.0,normal,normal,notpresent,notpresent,100.0,...,49.0,6600.0,5.4,no,no,no,good,no,no,notckd
398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,...,51.0,7200.0,5.9,no,no,no,good,no,no,notckd


In [112]:
df.dtypes

age               float64
bp                float64
sg                float64
al                float64
su                float64
rbc                object
pc                 object
pcc                object
ba                 object
bgr               float64
bu                float64
sc                float64
sod               float64
pot               float64
hemo              float64
pcv               float64
wc                float64
rc                float64
htn                object
dm                 object
cad                object
appet              object
pe                 object
ane                object
classification     object
dtype: object

In [117]:
col_dict = dict(df.dtypes)
numeric_vals = []
for col in col_dict:
    if str(col_dict[col]) == 'float64':
        numeric_vals.append(col)
        
numeric_vals

['age',
 'bp',
 'sg',
 'al',
 'su',
 'bgr',
 'bu',
 'sc',
 'sod',
 'pot',
 'hemo',
 'pcv',
 'wc',
 'rc']

In [119]:
for col in numeric_vals:
    renamed_df.loc[:,col] = pd.to_numeric(renamed_df[col],errors='coerce')
renamed_df.dtypes

age               float64
bp                float64
sg                float64
al                float64
su                float64
rbc                object
pc                 object
pcc                object
ba                 object
bgr               float64
bu                float64
sc                float64
sod               float64
pot               float64
hemo              float64
pcv               float64
wc                float64
rc                float64
htn                object
dm                 object
cad                object
appet              object
pe                 object
ane                object
classification     object
dtype: object

#### 3. Label Encoder

Encode labels with value between 0 and n_classes-1.

> from sklearn.preprocessing import LabelEncoder

In [130]:
# ANSWER
X = renamed_df.drop('classification',axis=1)
y = renamed_df.classification

In [134]:
y

0         ckd
1         ckd
2         ckd
3         ckd
4         ckd
        ...  
395    notckd
396    notckd
397    notckd
398    notckd
399    notckd
Name: classification, Length: 400, dtype: object

In [132]:
le = LabelEncoder()
le.fit_transform(y)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

#### 4. OneHotEncoder 

Encode categorical integer features as a one-hot numeric array.

The input to this transformer should be an array-like of integers or strings, denoting the values taken on by categorical (discrete) features. The features are encoded using a one-hot (aka ‘one-of-K’ or ‘dummy’) encoding scheme. This creates a binary column for each category and returns a sparse matrix or dense array.

By default, the encoder derives the categories based on the unique values in each feature. Alternatively, you can also specify the categories manually. The OneHotEncoder previously assumed that the input features take on values in the range [0, max(values)). This behaviour is deprecated.

This encoding is needed for feeding categorical data to many scikit-learn estimators, notably linear models and SVMs with the standard kernels.

> from sklearn.preprocessing import OneHotEncoder

In [140]:
# ANSWER
col_dict = dict(X.dtypes)
vals_to_encode = []
for col in col_dict:
    if str(col_dict[col]) == 'object':
        vals_to_encode.append(col)
        
vals_to_encode

['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane']

In [142]:
X.dtypes

age      float64
bp       float64
sg       float64
al       float64
su       float64
rbc       object
pc        object
pcc       object
ba        object
bgr      float64
bu       float64
sc       float64
sod      float64
pot      float64
hemo     float64
pcv      float64
wc       float64
rc       float64
htn       object
dm        object
cad       object
appet     object
pe        object
ane       object
dtype: object

In [145]:
ohe = OneHotEncoder()
ohe.fit_transform(X[vals_to_encode])
ohe.categories_

[array(['abnormal', 'normal'], dtype=object),
 array(['abnormal', 'normal'], dtype=object),
 array(['notpresent', 'present'], dtype=object),
 array(['notpresent', 'present'], dtype=object),
 array(['no', 'yes'], dtype=object),
 array(['no', 'yes'], dtype=object),
 array(['no', 'yes'], dtype=object),
 array(['good', 'poor'], dtype=object),
 array(['no', 'yes'], dtype=object),
 array(['no', 'yes'], dtype=object)]

In [152]:
ohe.fit_transform(X[vals_to_encode]).shape

(400, 20)

In [154]:
X.shape, len(vals_to_encode)

((400, 24), 10)

#### 5. Dummy Variables

Convert categorical variable into dummy/indicator variables

In [149]:
# ANSWER
column_trans = make_column_transformer(
                (OneHotEncoder(), vals_to_encode),
                remainder='passthrough'
                )

In [157]:
ct = column_trans.fit_transform(X)

In [161]:
ct_cols = column_trans.get_feature_names()

In [229]:
mapper = {}
for i in range(len(ct_cols)):
    mapper[i] = ct_cols[i]
    
i=0

for val in vals_to_encode:
    
    mapper[i] = '{feature}_{status}'.format(feature=val,status='no')
    i += 1
    mapper[i] = '{feature}_{status}'.format(feature=val,status='yes')
    i += 1
    
mapper

{0: 'rbc_no',
 1: 'rbc_yes',
 2: 'pc_no',
 3: 'pc_yes',
 4: 'pcc_no',
 5: 'pcc_yes',
 6: 'ba_no',
 7: 'ba_yes',
 8: 'htn_no',
 9: 'htn_yes',
 10: 'dm_no',
 11: 'dm_yes',
 12: 'cad_no',
 13: 'cad_yes',
 14: 'appet_no',
 15: 'appet_yes',
 16: 'pe_no',
 17: 'pe_yes',
 18: 'ane_no',
 19: 'ane_yes',
 20: 'age',
 21: 'bp',
 22: 'sg',
 23: 'al',
 24: 'su',
 25: 'bgr',
 26: 'bu',
 27: 'sc',
 28: 'sod',
 29: 'pot',
 30: 'hemo',
 31: 'pcv',
 32: 'wc',
 33: 'rc'}

In [230]:
ct_df = pd.DataFrame(ct)
ct_df = ct_df.rename(mapper=mapper,axis=1)
ct_df.T

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
rbc_no,0.00,0.00,0.00,0.000,0.00,0.000,0.00,0.000,0.000,1.00,...,0.000,0.000,0.00,0.000,0.00,0.00,0.000,0.00,0.000,0.000
rbc_yes,1.00,1.00,1.00,1.000,1.00,1.000,1.00,1.000,1.000,0.00,...,1.000,1.000,1.00,1.000,1.00,1.00,1.000,1.00,1.000,1.000
pc_no,0.00,0.00,0.00,1.000,0.00,0.000,0.00,1.000,1.000,1.00,...,0.000,0.000,0.00,0.000,0.00,0.00,0.000,0.00,0.000,0.000
pc_yes,1.00,1.00,1.00,0.000,1.00,1.000,1.00,0.000,0.000,0.00,...,1.000,1.000,1.00,1.000,1.00,1.00,1.000,1.00,1.000,1.000
pcc_no,1.00,1.00,1.00,0.000,1.00,1.000,1.00,1.000,0.000,0.00,...,1.000,1.000,1.00,1.000,1.00,1.00,1.000,1.00,1.000,1.000
pcc_yes,0.00,0.00,0.00,1.000,0.00,0.000,0.00,0.000,1.000,1.00,...,0.000,0.000,0.00,0.000,0.00,0.00,0.000,0.00,0.000,0.000
ba_no,1.00,1.00,1.00,1.000,1.00,1.000,1.00,1.000,1.000,1.00,...,1.000,1.000,1.00,1.000,1.00,1.00,1.000,1.00,1.000,1.000
ba_yes,0.00,0.00,0.00,0.000,0.00,0.000,0.00,0.000,0.000,0.00,...,0.000,0.000,0.00,0.000,0.00,0.00,0.000,0.00,0.000,0.000
htn_no,0.00,1.00,1.00,0.000,1.00,0.000,1.00,1.000,0.000,0.00,...,1.000,1.000,1.00,1.000,1.00,1.00,1.000,1.00,1.000,1.000
htn_yes,1.00,0.00,0.00,1.000,0.00,1.000,0.00,0.000,1.000,1.00,...,0.000,0.000,0.00,0.000,0.00,0.00,0.000,0.00,0.000,0.000


####  6. Set Target 

Set `classification` as target.

#### 4. Select Feature

The classes in the sklearn.feature_selection module can be used for feature selection/dimensionality reduction on sample sets, either to improve estimators’ accuracy scores or to boost their performance on very high-dimensional datasets.

##### 4.1 Univariate Selection

Univariate feature selection works by selecting the best features based on univariate statistical tests. It can be seen as a preprocessing step to an estimator. Scikit-learn exposes feature selection routines as objects that implement the transform method:

- SelectKBest removes all but the  highest scoring features
- Use sklearn.feature_selection.chi2 as score function
    > Recall that the chi-square test measures dependence between stochastic variables, so using this function “weeds out” the features that are the most likely to be independent of class and therefore irrelevant for classification.


More Reads:
[Univariate feature selection](https://scikit-learn.org/stable/modules/feature_selection.html)

- Create an instance of SelectKBest
    - Use sklearn.feature_selection.chi2 as score_func
    - Use k of your choice
- Fit X, y 
- Find top 4 features
- Transform features to a DataFrame

In [231]:
# ANSWER
skb = SelectKBest(score_func=chi2, k=4)
skb.fit(ct_df,y)
scores = skb.scores_

In [232]:
scores_df = pd.DataFrame({'Columns':ct_df.columns,'Scores':scores})
scores_df.sort_values(by='Scores',ascending=False)

,Columns,Scores
32,wc,25847.635652
26,bu,2341.230273
25,bgr,2092.920330
27,sc,357.989486
31,pcv,293.863135
23,al,216.000000
20,age,119.305673
30,hemo,101.643844
24,su,94.800000
9,htn_yes,88.200000


##### 4.2 Recursive feature elimination

Given an external estimator that assigns weights to features (e.g., the coefficients of a linear model), recursive feature elimination (RFE) is to select features by recursively considering smaller and smaller sets of features. First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through a coef_ attribute or through a feature_importances_ attribute. Then, the least important features are pruned from current set of features.That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.

More Reads:
[Recursive feature elimination](https://scikit-learn.org/stable/modules/feature_selection.html)

- Use RFE to extract feature
    - use LogisticRegression as estimator
    - Number of n_features_to_select as of your choice
- Fit X, y to RFE
- Find Selected Features

In [233]:
# ANSWER
lr = LogisticRegression(max_iter=4000,n_jobs=-1)
rfe = RFE(lr, n_features_to_select=4)
rfe.fit(ct_df,y)

RFE(estimator=LogisticRegression(max_iter=4000, n_jobs=-1),
    n_features_to_select=4)

In [234]:
rfe_scores_df = pd.DataFrame({'Columns':ct_df.columns,'Scores':scores,'Selection':rfe.ranking_})
rfe_scores_df.sort_values('Selection')

,Columns,Scores,Selection
27,sc,357.989486,1
23,al,216.000000,1
9,htn_yes,88.200000,1
10,dm_no,42.819011,1
15,appet_yes,49.200000,2
11,dm_yes,82.200000,3
16,pe_no,10.696296,4
14,appet_no,12.686792,5
17,pe_yes,45.600000,6
30,hemo,101.643844,7


#### Create multiple Classifier Model

Create multiple classifier models to predict the chronic kidney disease. Use any models of your choice. Evaluate all models and select the best model according to their performance. 

In [235]:
### K-Best
cvs = cross_val_score(lr, ct_df[['wc','bu','bgr','sc']], y, cv=5, n_jobs= -1)
cvs.mean()

0.8425

In [236]:
### rfe on lr
cvs = cross_val_score(lr, ct_df[['sc','al', 'htn_yes', 'dm_no']], y, cv=5, n_jobs= -1)
cvs.mean()

0.95



---



---



> > > > > > > > > © 2019 Institute of Data


---



---



